In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def criar_carteira_comparar(ativos, indices, data_comparacao):
    """
    Cria uma carteira com base nos ativos e compara sua performance com índices de mercado.

    Parâmetros:
        ativos (dict): Um dicionário onde as chaves são os tickers dos ativos 
                       e os valores são listas no formato [peso].
        indices (list): Lista de tickers de índices a serem comparados.
        data_comparacao (tuple): Datas de início e fim para a comparação, no formato ('AAAA-MM-DD', 'AAAA-MM-DD').

    Retorno:
        pd.DataFrame: DataFrame contendo o rendimento acumulado da carteira e dos índices.
    """
    # Validar as datas
    data_inicio = pd.to_datetime(data_comparacao[0])  # Data de início definida pelo usuário
    data_fim = pd.to_datetime(data_comparacao[1])

    # Baixar os dados de mercado
    tickers = list(ativos.keys()) + indices
    dados = yf.download(tickers, start=data_inicio, end=data_fim)['Adj Close']

    # Preencher valores ausentes
    dados = dados.fillna(method='ffill').fillna(method='bfill')

    # Calcular retornos diários
    retornos_diarios = dados.pct_change().dropna()

    # Calcular pesos da carteira
    pesos_carteira = pd.Series({ticker: ativos[ticker][0] for ticker in ativos}).rename_axis("Ativos")
    pesos_carteira /= pesos_carteira.sum()

    # Calcular o retorno da carteira
    retorno_carteira = (retornos_diarios[list(ativos.keys())] @ pesos_carteira).rename("Carteira")
    rendimento_acumulado = (1 + retorno_carteira).cumprod() - 1

    # Calcular o rendimento acumulado dos índices
    rendimento_indices = (1 + retornos_diarios[indices]).cumprod() - 1

    # Combinar os resultados em um único DataFrame
    resultado = pd.concat([rendimento_acumulado, rendimento_indices], axis=1)
    resultado.index.name = "Data"

    # Criar o gráfico
    fig = go.Figure()

    # Adicionar a performance da carteira
    fig.add_trace(go.Scatter(
        x=resultado.index,
        y=resultado['Carteira'],
        mode='lines',
        name='Carteira',
        line=dict(width=2, color='blue')
    ))

    # Adicionar a performance dos índices
    for indice in indices:
        fig.add_trace(go.Scatter(
            x=resultado.index,
            y=resultado[indice],
            mode='lines',
            name=indice,
            line=dict(width=2)
        ))

    # Configurar o layout do gráfico
    fig.update_layout(
        title="Performance Acumulada: Carteira vs Índices",
        xaxis_title="Data",
        yaxis_title="Rendimento Acumulado",
        template="plotly_white",
        legend_title="Legenda",
    )

    # Exibir o gráfico
    fig.show()

    return resultado


In [3]:
# USANDO EM CARTEIRA EM CENÁRIO EXATO DE 24/11/2024
#GUARDE ESSA CONFIGURAÇÃO PARA VER O PROGRESSO QUE ESSE COMPONENTE DA CARTEIRA TERÁ SOZINHO NOS PRÓXIMOS ANOS
start = '2018-01-01'
ativos = {
    'BBAS3.SA': [ 0.209],  
    'CSNA3.SA': [ 0.042],  
    'FLRY3.SA': [  0.051],
    'ITSA4.SA': [ 0.089],
    'KEPL3.SA': [ 0.175],
    'KLBN4.SA': [ 0.083],
    'TAEE4.SA': [ 0.061],
    'MGLU3.SA': [ 0.002],
    'UNIP6.SA': [ 0.107],
    'GOAU4.SA': [ 0.18],
 
}

indices = ['^GSPC', '^BVSP','^N225']  
data_comparacao = (start, '2024-11-20') # datas de compra precisam ser antes da primeira data desse intervalo

resultado = criar_carteira_comparar(ativos, indices, data_comparacao)



[*********************100%%**********************]  13 of 13 completed
C:\Users\OtavioDaflon\AppData\Local\Temp\ipykernel_33680\1890222411.py:28: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [ ]:
# total = 33352
# aloc = 1697.22
# print((aloc/total)*100)

5.0888102662509


# Com n Carteiras

In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

def criar_comparacao_carteiras(indices, data_comparacao, *lista_carteiras):
    """
    Compara o desempenho de múltiplas carteiras com índices de mercado.

    Parâmetros:
        indices (list): Lista de tickers de índices a serem comparados.
        data_comparacao (tuple): Datas de início e fim no formato ('AAAA-MM-DD', 'AAAA-MM-DD').
        *lista_carteiras (dict): Dicionários com os ativos e pesos de cada carteira no formato {ticker: [data_compra, peso]}.

    Retorno:
        pd.DataFrame: DataFrame com o rendimento acumulado de todas as carteiras e dos índices.
    """
    # Determinar a data de início mais recente entre todas as carteiras
    datas_compra = []
    for carteira in lista_carteiras:
        datas_compra += [pd.to_datetime(carteira[ticker][0]) for ticker in carteira]
    data_inicio = max(datas_compra)
    data_fim = pd.to_datetime(data_comparacao[1])
    
    if pd.to_datetime(data_comparacao[0]) < data_inicio:
        raise ValueError(f"A data de início não pode ser anterior a {data_inicio.date()}.")

    # Baixar os dados de mercado
    tickers = [ticker for carteira in lista_carteiras for ticker in carteira.keys()] + indices
    dados = yf.download(tickers, start=data_inicio, end=data_fim)['Adj Close']
    
    # Preencher valores ausentes
    dados = dados.fillna(method='ffill').fillna(method='bfill')
    
    # Calcular os retornos diários
    retornos_diarios = dados.pct_change().dropna()

    def calcular_rendimento(carteira, retornos):
        # Calcular os pesos da carteira
        pesos = {ticker: carteira[ticker][1] for ticker in carteira}
        pesos_carteira = pd.Series(pesos)
        pesos_carteira /= pesos_carteira.sum()
        # Calcular o retorno da carteira
        retorno_carteira = (retornos[list(carteira.keys())] @ pesos_carteira).rename("Carteira")
        rendimento_acumulado = (1 + retorno_carteira).cumprod() - 1
        return rendimento_acumulado

    # Calcular o rendimento acumulado para cada carteira
    resultados_carteiras = {}
    for i, carteira in enumerate(lista_carteiras, start=1):
        resultados_carteiras[f'Carteira{i}'] = calcular_rendimento(carteira, retornos_diarios)
    
    # Calcular o rendimento acumulado dos índices
    rendimento_indices = (1 + retornos_diarios[indices]).cumprod() - 1

    # Consolidar resultados
    resultado = pd.concat(
        [pd.DataFrame(resultados_carteiras), rendimento_indices],
        axis=1
    )
    resultado.index.name = "Data"

    # Criar o gráfico
    fig = go.Figure()

    # Adicionar as carteiras ao gráfico
    for i, carteira_nome in enumerate(resultados_carteiras.keys(), start=1):
        fig.add_trace(go.Scatter(
            x=resultado.index,
            y=resultado[carteira_nome],
            mode='lines',
            name=carteira_nome,
            line=dict(width=2)
        ))

    # Adicionar os índices ao gráfico
    for indice in indices:
        fig.add_trace(go.Scatter(
            x=resultado.index,
            y=resultado[indice],
            mode='lines',
            name=indice,
            line=dict(width=2, dash='dot')
        ))

    # Configurar layout do gráfico
    fig.update_layout(
        title="Performance Acumulada: Carteiras vs Índices",
        xaxis_title="Data",
        yaxis_title="Rendimento Acumulado",
        template="plotly_white",
        legend_title="Legenda",
    )
    
    fig.show()
    
    return resultado


In [13]:
start = '2021-07-01'
ativos = {
    'BBAS3.SA': [start, 0.209],  
    'CSNA3.SA': [start, 0.042],  
    'FLRY3.SA': [start,  0.051],
    'ITSA4.SA': [start, 0.089],
    'KEPL3.SA': [start, 0.175],
    'KLBN4.SA': [start, 0.083],
    'TAEE4.SA': [start, 0.061],
    'MGLU3.SA': [start, 0.002],
    'UNIP6.SA': [start, 0.107],
    'GOAU4.SA': [start, 0.18],
 
}
perpectiva = {
    'LFTS11.SA': [start,0.25],
    'BBAS3.SA':[start,0.07],
    'CEAB3.SA':[start,0.03],
    'CMIN3.SA':[start,0.04],
    'JBSS3.SA':[start,0.05],
    'KLBN11.SA':[start,0.06],
    'MRFG3.SA':[start,0.04],
    'PETR4.SA':[start,0.05],
    'PSSA3.SA':[start,0.07],
    'SUZB3.SA':[start,0.05],
    'VALE3.SA':[start,0.07],
    'IVVB11.SA':[start,0.04],
    'NASD11.SA':[start,0.02],
    'GOLD11.SA':[start,0.08],
    'HASH11.SA':[start,0.04],
    'XINA11.SA':[start,0.04]
}
smallblue3 = {
    'BLAU3.SA':[start,0.08],
    'BRAV3.SA':[start,0.05],
    'CEAB3.SA':[start,0.14],
    'CSMG3.SA':[start,0.12],
    'CYRE3.SA':[start,0.09],
    'DIRR3.SA':[start,0.11],
    'MDNE3.SA':[start,0.08],
    'PLPL3.SA':[start,0.07],
    'POMO4.SA':[start,0.11]
}
recomrico = {
    'BBAS3.SA':[start,0.1],
    'BRSR6.SA':[start,0.1],
    'BRAP4.SA':[start,0.1],
    'EVEN3.SA':[start,0.1],
    'JHSF3.SA':[start,0.1],
    'LAVV3.SA':[start,0.1],
    'PETR4.SA':[start,0.1],
    'RECV3.SA':[start,0.1],
    'STBP3.SA':[start,0.1],
    'TIMS3.SA':[start,0.1],

}
test = {
    'PETR3.SA':[start,1]
}

indices = ['^GSPC', '^BVSP','^N225']  
data_comparacao = (start, '2024-11-20') # datas de compra precisam ser antes da primeira data desse intervalo
wallet_list=[ativos,perpectiva,smallblue3,recomrico,test]

resultado = criar_comparacao_carteiras( indices, data_comparacao,*wallet_list)

[*********************100%%**********************]  45 of 45 completed
C:\Users\OtavioDaflon\AppData\Local\Temp\ipykernel_21576\3932832409.py:32: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

